In [1]:
import sys
import os as os
import numpy as np
from scipy import misc
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#import cv2
import h5py
import copy
from keras.optimizers import SGD
from keras import callbacks as cb
from keras.layers import Input, merge
from keras.layers.convolutional import Convolution2D, MaxPooling2D
from keras.layers.core import Dropout, Activation
from keras.layers.pooling import GlobalAveragePooling2D
from keras.models import Model
import time
sys.path.append(os.path.abspath('squeezeNet'))
from squeezenet import *

%load_ext autoreload
%autoreload 2



Using Theano backend.
WARNING (theano.sandbox.cuda): The cuda backend is deprecated and will be removed in the next release (v0.10).  Please switch to the gpuarray backend. You can get more information about how to switch at this URL:
 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

 https://github.com/Theano/Theano/wiki/Converting-to-the-new-gpu-back-end%28gpuarray%29

Using gpu device 0: Tesla M60 (CNMeM is enabled with initial size: 95.0% of memory, cuDNN 5110)


In [2]:
## this code creates the ground truth images, and the corresponding heatMaps, aquired by hand labelling
## of the training data. 
## the input to my net will be an image of 720x1080x3
## the output is a heatMap of 16x27

projectPath = '/home/solomond78/Desktop/GIT/FishProject1/fish_keras'
dataPath = '/home/solomond78/Desktop/FishData/train'
scorePath = os.path.join('/home/solomond78/Desktop/FishData', 'binaryScoreMaps')
print scorePath
# run through binaryScoreMap folders and count how many are there
fishTypes = os.listdir(scorePath)
print "fishTypes : " + fishTypes[0]
numOfImages = 0
for fishIdx in np.arange(len(fishTypes)):
    fishPath = os.path.join(scorePath, fishTypes[fishIdx])
    # filter all non jpg files\folders
    imageNames = [fn for fn in os.listdir(fishPath) if fn.endswith("jpg")]
    numOfImages = numOfImages + len(imageNames)
    
print 'number of score maps : ', numOfImages
h_score = 16 #720
h_img = 720
w_score = 27 #1080
w_img = 1080

# create an array of all scoreMaps & all images
scoreMaps = np.zeros((h_score,w_score,numOfImages))
images = np.zeros((numOfImages,3, h_img, w_img), dtype=np.float32)

# run through the data, and save all of it within an array
counter = 0
for fishIdx in np.arange(len(fishTypes)):
    fishPath = os.path.join(scorePath, fishTypes[fishIdx])
    # filter all non jpg files\folders
    scoreMapNames = [fn for fn in os.listdir(fishPath) if fn.endswith("jpg")]
    # run through the images and perform 2 things:
    # 1. resize the score images to be of size h_score x w_score
    # 2. perform all required data manipulation on the images themselfs..
    # save all...
    for imageIdx in np.arange(len(scoreMapNames)):
        # print the number of image
        #print 'image number : ' + np.str(counter) + '/' + np.str(numOfImages)
        # everything about the image
        imagePath = os.path.join(dataPath, fishTypes[fishIdx], scoreMapNames[imageIdx])
        img = misc.imread(imagePath).astype(np.float32)
        img = misc.imresize(img, (720, 1080)).astype(np.float32)
        #plt.imshow(img)        
        aux = copy.copy(img)
        img[:, :, 0] = aux[:, :, 2]
        img[:, :, 2] = aux[:, :, 0]
        # Remove train image mean
        img[:, :, 0] -= 104.006
        img[:, :, 1] -= 116.669
        img[:, :, 2] -= 122.679
        img = np.transpose(img, (2, 0, 1))
        img = np.expand_dims(img, axis=0)
        images[counter,:,:,:] = img
        
        #everything about the scoreMap
        scoreMapPath = os.path.join(fishPath, scoreMapNames[imageIdx])
        scoreMap = misc.imread(scoreMapPath, 'L').astype(np.float32)
        scoreMap = misc.imresize(scoreMap, (h_score,w_score)).astype(np.bool)
        scoreMaps[:,:, counter] = scoreMap
        counter += 1
        

/home/solomond78/Desktop/FishData/binaryScoreMaps
fishTypes : LAG
number of score maps :  3013


In [3]:
np.random.seed(1)
np.random.shuffle(images)
np.random.seed(1)
scoreMaps = np.transpose(scoreMaps, (2,0,1))
np.random.shuffle(scoreMaps)
scoreMaps = np.transpose(scoreMaps, (1,2,0))

In [4]:
from keras.preprocessing import image as imageClass

In [11]:
print np.floor(0.9*np.size(images,0))
print transposedScoreMaps.shape

2711.0
(3013, 2, 16, 27)


In [ ]:
# save the array
import h5py
np.save("/home/solomond78/Desktop/FishData/myFile", images)
np.save("/home/solomond78/Desktop/FishData/myGroundTruth", scoreMaps)
#images = np.load("C:\\Users\\David\\Desktop\\Fish\\train\\myFile.npy")
#scoreMaps = np.load("C:\\Users\\David\\Desktop\\Fish\\train\\myGroundTruth.npy")

In [5]:
# try training the network
model = get_squeezenet(2,720,1080, dim_ordering='th')

#model.summary()
extendedScoreMaps = np.zeros((2,scoreMaps.shape[0],scoreMaps.shape[1],scoreMaps.shape[2]))
extendedScoreMaps[0,:,:,:] = 1*np.expand_dims(scoreMaps, axis=0).astype(np.bool)
theNumberOne = np.ones((1,1,1,1))
extendedScoreMaps[1,:,:,:] = np.abs(extendedScoreMaps[0,:,:,:]-theNumberOne)
#print extendedScoreMaps.shape

transposedScoreMaps = np.transpose(extendedScoreMaps, (3, 0, 1,2))

datagen = imageClass.ImageDataGenerator(
            horizontal_flip = True,
            vertical_flip = True)
print 'ready to begin training'

ready to begin training


In [6]:
#sgd = SGD(lr=0.003, decay=1e-6, momentum=0.9, nesterov=True)
#earlyStop = cb.EarlyStopping(monitor='val_acc', min_delta=0.01, patience=2, mode='max')
checkPoint = cb.ModelCheckpoint('/home/solomond78/Desktop/FishData/kerasModel/model_weights.{epoch:02d}-{val_acc:.2f}.h5', monitor='val_acc', verbose=1, save_best_only=True, save_weights_only=False, mode='max', period=1)
reduce_lr = cb.ReduceLROnPlateau(monitor='acc', factor=0.5, patience=1, min_lr=0.00001)


model.compile(loss="binary_crossentropy", optimizer='sgd',
              metrics=['accuracy', 'recall'])
model.load_weights(os.path.abspath('squeezeNet/model/squeezenet_weights_th_dim_ordering_th_kernels.h5'), by_name=True)
#model.load_weights('/home/solomond78/Desktop/FishData/kerasModel/thirdModel_epoch0.h5', by_name=True)
numOfTraining = np.floor(0.9*np.size(images,0))
datagen.fit(images[0:numOfTraining,:,:,:])
model.fit(images, transposedScoreMaps, batch_size=18, validation_split=0.1,shuffle=True,
                   callbacks=[checkPoint, earlyStop, reduce_lr],
                   #validation_data=(images[numOfTraining+1:,:,:,:], transposedScoreMaps[numOfTraining+1:,:,:,:]),
                   class_weight=np.array([1.6,0.4]), initial_epoch=7
                   verbose=1,  nb_epoch=15)


Train on 2711 samples, validate on 302 samples
Epoch 1/15
2711/2711 [==============================] - 187s - loss: 0.3108 - acc: 0.8938 - recall: 0.8938 - val_loss: 0.2553 - val_acc: 0.9030 - val_recall: 0.9030
Epoch 2/15
2711/2711 [==============================] - 186s - loss: 0.2563 - acc: 0.9001 - recall: 0.9001 - val_loss: 0.2433 - val_acc: 0.9124 - val_recall: 0.9124
Epoch 3/15
2711/2711 [==============================] - 186s - loss: 0.2353 - acc: 0.9061 - recall: 0.9061 - val_loss: 0.2406 - val_acc: 0.9122 - val_recall: 0.9122
Epoch 4/15
2711/2711 [==============================] - 186s - loss: 0.2199 - acc: 0.9121 - recall: 0.9121 - val_loss: 0.2029 - val_acc: 0.9219 - val_recall: 0.9219
Epoch 5/15
2711/2711 [==============================] - 187s - loss: 0.2095 - acc: 0.9161 - recall: 0.9161 - val_loss: 0.1945 - val_acc: 0.9204 - val_recall: 0.9204
Epoch 6/15
2711/2711 [==============================] - 186s - loss: 0.1946 - acc: 0.9213 - recall: 0.9213 - val_loss: 0.1840 - 

In [11]:
model.fit(images, transposedScoreMaps, batch_size=18, validation_split=0.1,shuffle=True,
                   callbacks=[checkPoint, reduce_lr],
                   #validation_data=(images[numOfTraining+1:,:,:,:], transposedScoreMaps[numOfTraining+1:,:,:,:]),
                   class_weight=np.array([1.6,0.4]), initial_epoch=30,
                   verbose=1,  nb_epoch=40)

Train on 2711 samples, validate on 302 samples
Epoch 31/40
2711/2711 [==============================] - 187s - loss: 0.1120 - acc: 0.9542 - recall: 0.9542 - val_loss: 0.1220 - val_acc: 0.9510 - val_recall: 0.9510
Epoch 32/40
2711/2711 [==============================] - 186s - loss: 0.1117 - acc: 0.9544 - recall: 0.9544 - val_loss: 0.1216 - val_acc: 0.9511 - val_recall: 0.9511
Epoch 33/40
2711/2711 [==============================] - 186s - loss: 0.1115 - acc: 0.9542 - recall: 0.9542 - val_loss: 0.1364 - val_acc: 0.9459 - val_recall: 0.9459
Epoch 34/40
2711/2711 [==============================] - 187s - loss: 0.1091 - acc: 0.9554 - recall: 0.9554 - val_loss: 0.1178 - val_acc: 0.9531 - val_recall: 0.9531
Epoch 35/40
2711/2711 [==============================] - 187s - loss: 0.1085 - acc: 0.9555 - recall: 0.9555 - val_loss: 0.1249 - val_acc: 0.9512 - val_recall: 0.9512
Epoch 36/40
2711/2711 [==============================] - 187s - loss: 0.1067 - acc: 0.9561 - recall: 0.9561 - val_loss: 0.1

extendedScoreMaps = np.zeros((2,scoreMaps.shape[0],scoreMaps.shape[1],scoreMaps.shape[2]))
extendedScoreMaps[0,:,:,:] = 1*np.expand_dims(scoreMaps, axis=0).astype(np.bool)
#extendedScoreMaps = 1*extendedScoreMaps.astype(np.bool)
theNumberOne = np.ones((1,1,1,1))
extendedScoreMaps[1,:,:,:] = np.abs(extendedScoreMaps[0,:,:,:]-theNumberOne)
print extendedScoreMaps.shape

In [69]:
model.fit_generator(datagen.flow(images[0:numOfTraining,:,:,:], transposedScoreMaps[0:numOfTraining,:,:,:], batch_size=18),
                   callbacks=[checkPoint, earlyStop, reduce_lr],
                   #validation_data=(images[numOfTraining+1:,:,:,:], transposedScoreMaps[numOfTraining+1:,:,:,:]),
                   class_weight=np.array([1.6,0.4]),
                   verbose=1, samples_per_epoch=np.int(numOfTraining), nb_epoch=15)
extendedScoreMaps.shape
theNumberOne = np.ones((1,1,1,1))
theNumberOne.shape
extendedScoreMaps-theNumberOne
for epochNum in np.arange(10):
    model.fit(x=images, y=transposedScoreMaps, batch_size=18, nb_epoch=epochNum+1, verbose=1, callbacks=None, validation_split=0.1, shuffle=True, class_weight=np.array([1.5,0.5]), sample_weight=None, initial_epoch=epochNum)
    model.save('/home/solomond78/Desktop/FishData/kerasModel/thirdModel_epoch' + np.str(epochNum) + '.h5')

array([[[[-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         ..., 
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.]],

        [[-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         ..., 
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.]],

        [[-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         ..., 
         [-1., -1., -1., ..., -1., -1.,  0.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.]],

        ..., 
        [[-1., -1., -1., ..., -1., -1., -1.],
         [-1., -1., -1., ..., -1., -1., -1.],
         [-1., 